# Summary
This document corresponds to Exercise 5 of [this file](https://github.com/PerformanceEstimation/Learning-Performance-Estimation/blob/main/Exercises/Course.pdf).

The first step consists in installing [PEPit](https://pypi.org/project/PEPit/) and its dependencies.

In [ ]:
!pip install pepit

In [ ]:
from PEPit import PEP
from PEPit.functions import SmoothConvexFunction
from PEPit.functions import ConvexIndicatorFunction
from PEPit.primitive_steps import proximal_step
from PEPit.primitive_steps import linear_optimization_step


def wc_projected_gradient(L, D, gamma, n, verbose=1):

    # Instantiate PEP
    problem = PEP()

    # Declare a convex smooth function and a closed convex proper indicator
    f1 = problem.declare_function(SmoothConvexFunction, L=L)
    f2 = problem.declare_function(ConvexIndicatorFunction, D=D)
    func = f1 + f2

    # Start by defining its unique optimal point xs = x_*
    xs = func.stationary_point()

    # Then define the starting point x0 of the algorithm
    x0 = problem.set_initial_point()
    # Enforce the feasibility of x0 : there is no initial constraint on x0 (this is not mandatory)
    _ = f1(x0)
    _ = f2(x0)

    # Run the projected gradient method starting from x0
    x = x0
    for _ in range(n):
        y = x - gamma * f1.gradient(x)
        x, _, _ = proximal_step(y, f2, gamma)

    # Set the performance metric to the distance between x and xs
    problem.set_performance_metric(f1(x)-f1(xs))

    # Solve the PEP
    pepit_tau = problem.solve(verbose)
    
    # Return the worst-case guarantee of the evaluated method
    return pepit_tau

def wc_frank_wolfe(L, D, n, verbose=1):

    # Instantiate PEP
    problem = PEP()

    # Declare a smooth convex function and a convex indicator of rayon D
    f1 = problem.declare_function(function_class=SmoothConvexFunction, L=L)
    f2 = problem.declare_function(function_class=ConvexIndicatorFunction, D=D)
    # Define the function to optimize as the sum of func1 and func2
    f = f1 + f2

    # Start by defining its unique optimal point xs = x_* and its function value fs = F(x_*)
    xs = f.stationary_point()
    fs = f(xs)

    # Then define the starting point x0 of the algorithm and its function value f0
    x0 = problem.set_initial_point()

    # Enforce the feasibility of x0 : there is no initial constraint on x0
    _ = f1(x0)
    _ = f2(x0)

    # Compute n steps of the Conditional Gradient / Frank-Wolfe method starting from x0
    x = x0
    for i in range(n):
        g = f1.gradient(x)
        y, _, _ = linear_optimization_step(g, f2)
        lam = 2 / (i + 2)
        x = (1 - lam) * x + lam * y

    # Set the performance metric to the final distance in function values to optimum
    problem.set_performance_metric(f(x) - fs)

    # Solve the PEP
    pepit_tau = problem.solve(verbose)
    
    # Return the worst-case guarantee of the evaluated method
    return pepit_tau